# Normalizar Columnas de la Agenda

In [1]:
import pandas as pd
import numpy as np
import datetime
import io
import re

In [2]:
archivo_excel = "C:/Users/contr/OneDrive/Desarrollo Diario/00. Matrices BDD/02. Agendamiento\Repositorio 2 Años/Agenda2018.xlsx"

nombre_hoja = 'Agenda2018'

# Lee el archivo de Excel en un DataFrame de pandas
df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja)

## Normalizar la Data (Agenda)

In [3]:
nuevos_nombres = ["fecha_reserva","marca","trabajo_agenda","patente","box","trabajo","idr","detalle","submodelo","teleoperador_7d","oficina","fecha_estado","canal","estado","vigencia","teleoperador_agendo","rut_cliente","nombre_cliente","fono_casa_cliente","fono_oficina_cliente","fono_celular_cliente","mail_cliente","vin","hora_inicio","ano","oficina_idr","patente_idr"]
df.columns = nuevos_nombres

In [4]:
df = df.astype(str)
df["fecha_reserva"] = pd.to_datetime(df["fecha_reserva"], format="%Y-%m-%d")
df["fecha_estado"] = pd.to_datetime(df["fecha_estado"], format="%Y-%m-%d")

In [5]:
df['vin'] = df['vin'].str.replace('O', '0')

In [6]:
df['vin'] = df['vin'].astype(str)

def clean_vin(vin):
    if len(vin) != 17:
        return ''
    return vin

df['vin'] = df['vin'].apply(clean_vin)

In [7]:
df['patente'] = df['patente'].astype(str)

def clean_patente(patente):
    if len(patente) != 7:
        return ''
    return patente

df['patente'] = df['patente'].apply(clean_patente)

In [8]:
df.replace("", np.nan, inplace=True)
#df['vin'] = df.groupby('patente')['vin'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

In [9]:
##ESTE ES
#df.replace("", np.nan, inplace=True)
df['vin'] = df.sort_values('fecha_reserva', ascending=False).groupby('patente')['vin'].transform('first')

In [10]:
df['vin_patente'] = np.where(df['vin'] == "", df['patente'], df['vin'])

In [11]:
df['oficina'] = df['oficina'].str.upper()
def asignar_unidad_negocios(oficina):
    if "NISSAN" in oficina:
        return "NISSAN"
    elif "TOYOTA" in oficina:
        return "TOYOTA"
    elif "DYP" in oficina:
        return "DYP"
    else:
        return "CHEVROLET"

df["unidad_negocios"] = df["oficina"].apply(asignar_unidad_negocios)

In [12]:

# Convertimos la columna 'fecha_reserva' a tipo datetime
df['fecha_reserva'] = pd.to_datetime(df['fecha_reserva'], format='%Y-%m-%d')

# Calculamos la fecha actual
today = pd.Timestamp.today()

# Agregamos una nueva columna 'ciclo'
df['ciclo'] = pd.cut(
    x=(today - df['fecha_reserva']).dt.days / 30,  # Calculamos la diferencia en meses
    bins=[-float('inf'), 0, 9, 18, 27, float('inf')],  # Definimos los límites de cada ciclo
    labels=['Ciclo 0', 'Primer Ciclo', 'Segundo Ciclo', 'Tercer Ciclo', 'Cuarto Ciclo']  # Definimos las etiquetas de cada ciclo
)

# Agregamos una condición para los casos en que la fecha de reserva es posterior a la fecha actual
df.loc[df['fecha_reserva'] > today, 'ciclo'] = 'Ciclo 0'


In [13]:
df.sort_values("fecha_reserva", ascending=False, inplace=True)
df_ultimoEstado = df.groupby("vin_patente").first()
df_ultimoEstado = df_ultimoEstado.astype(str)
df_ultimoEstado["fecha_reserva"] = pd.to_datetime(df_ultimoEstado["fecha_reserva"], format="%Y-%m-%d")
df_ultimoEstado["fecha_estado"] = pd.to_datetime(df_ultimoEstado["fecha_estado"], format="%Y-%m-%d")
df_ultimoEstado.to_excel("UltimoEstadoAgendaLargaALL.xlsx", index=False)